# Реализация модели защищенного канала передачи данных по спецификации TLS 1.3

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

----

In [3]:
# Криптография из различных ГОСТ'ов
from pygost import mgm
from pygost import gost3412
from pygost import gost3410
from pygost import gost34112012256

# Криптография из западных стандартов
from Crypto.Cipher import AES
from Crypto.Hash import SHA256

# HKDF
from Crypto.Protocol.KDF import HKDF

# КГПСЧ
from Crypto.Random import get_random_bytes, random
from Crypto.Util.number import getStrongPrime

# Различные утилиты
from collections import namedtuple
from Crypto.Util import number
from functools import partial
from binascii import hexlify
from enum import Enum
import uuid
import gmpy2
import pickle

----

## Реализация оберток над криптографическими алгоритмами

In [4]:
def multiply_by_scalar(curve, scalar, point):
    """
    Умножение точки эллиптической кривой на скаляр 
    """
    
    return curve.exp(scalar, *point)


def add_points(curve, lhs, rhs):
    """
    Сложение двух точек эллиптической кривой
    """
    
    return curve._add(*lhs, *rhs)

---

In [5]:
#
# Сертификат это простая структурка, поэтому обойдусь именованным кортежем
#

Certificate = namedtuple('Certificate', ['id', 'public_key', 'signature'])

----

In [6]:
class KuznyechikMGM(object):
    """
    Класс-обертка над Кузнечиком в режиме MGM
    """
    
    KEY_SIZE = 32
    
    def __init__(self, key: bytes):
        """
        Инициализация - сохраняю ключ
        """

        self._key = key
    
    
    def encrypt(self, plaintext: bytes):
        """
        Зашифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = partial(KuznyechikMGM._encrypt_block, self._key)
        cipher = mgm.MGM(encrypter, KuznyechikMGM._block_size())
        
        nonce = KuznyechikMGM._generate_nonce()
        ct_with_tag = cipher.seal(nonce, plaintext, b'')
        
        return nonce, ct_with_tag[:-cipher.tag_size], ct_with_tag[-cipher.tag_size:]
    
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes):
        """
        Расшифрование шифром Кузнечик в режиме MGM
        """
        
        encrypter = partial(KuznyechikMGM._encrypt_block, self._key)
        cipher = mgm.MGM(encrypter, KuznyechikMGM._block_size())
            
        ct_with_tag = ciphertext + tag
            
        return cipher.open(nonce, ct_with_tag, b'')
        
        
    @staticmethod
    def _block_size() -> int:
        return gost3412.GOST3412Kuznechik.blocksize
        
        
    @staticmethod
    def _encrypt_block(key: bytes, block: bytes):
        cipher = gost3412.GOST3412Kuznechik(key)
        return cipher.encrypt(block)
    
    
    @staticmethod
    def _generate_nonce() -> bytes:
        nonce = get_random_bytes(KuznyechikMGM._block_size())
        while bytearray(nonce)[0] & 0x80 > 0:
            nonce = get_random_bytes(KuznyechikMGM._block_size())
            
        return nonce

In [7]:
class AESGCM(object):
    """
    Класс-обертка над AES в режиме GCM
    """
    
    KEY_SIZE = 16
    
    def __init__(self, key: bytes):
        """
        Инициализация - сохраняю ключ
        """

        self._key = key
        
        
    def encrypt(self, plaintext: bytes):
        """
        Зашифрование шифром AES в режиме GCM
        """
        
        cipher = AES.new(self._key, AES.MODE_GCM)
        return cipher.nonce, *cipher.encrypt_and_digest(plaintext)
    
    
    def decrypt(self, nonce: bytes, ciphertext: bytes, tag: bytes):
        """
        Зашифрование шифром AES в режиме GCM
        """
        
        cipher = AES.new(self._key, AES.MODE_GCM, nonce=nonce)
        return cipher.decrypt_and_verify(ciphertext, tag)

----

In [8]:
class Streebog(gost34112012256.GOST34112012256):
    """
    pycryptodome-совместимая обертка над хэш-функцией Стрибог
    """
    
    digest_size = 32
    
    def __init__(self, data=None):
        """
        Инициализация и обновление состояния (если надо)
        """
        
        super(Streebog, self).__init__()
        if data is not None:
            self.update(data)
    
    @staticmethod
    def new(data=None):
        """
        Создание нового объекта
        """
        
        return Streebog(data)

In [9]:
def hash_digest(hasher_type, *args) -> bytes:
    """
    Вычисление хэша при помощи объекта-обертки
    """
    
    hasher = hasher_type.new()
    
    for arg in args:
        hasher.update(arg)
        
    return hasher.digest()

----

In [10]:
class DiffieHellmanProcessor(object):
    """
    Обработчик обмена ключами по протоколу Диффи-Хеллмана
    """
    
    def __init__(self, group=None, generator=None):
        """
        Инициализация - выбор группы, образующего элемента
        Возможно конструирование по готовой группе и образующему
        """

        self._group = Integers(getStrongPrime(512)) if group is None else group
        self._generator = random.randint(int(2), int(self._group.order() - 2)) if generator is None \
            else generator
        
        self._modulus = self._group.order()
        
        trace('[DiffieHellmanProcessor]', f'Using group of order {self._modulus}')
        
    
    def generate_private_key(self):
        """
        Генерирование закрытого ключа
        """
        
        return random.randint(int(2), int(self.order - 2))
    
    
    def get_public_key(self, private_key):
        """
        Получение открытого ключа по закрытому
        """
        
        return self.exponentiate(private_key, self._generator)
    
    
    def exponentiate(self, scalar, element):
        """
        Возведение в степень элемента группы
        """
        
        return self._group(int(gmpy2.powmod(int(element), int(scalar), self.order)))
    
    
    def element_to_bytes(self, element):
        """
        Конвертация элемента группы в байтовое представление
        """
        
        return number.long_to_bytes(int(element))
    
    
    @property
    def group(self):
        """
        Получение группы
        """
        
        return self._group
    
        
    @property
    def generator(self):
        """
        Получение образующего элемента
        """
        
        return self._generator
    
    
    @property
    def order(self):
        """
        Получение порядка группы
        """
        
        return self._modulus

In [11]:
class EllipticCurveDiffieProcessor(object):
    """
    Обработчик обмена ключами по протоколу Диффи-Хеллмана
    в группе точек эллиптической кривой
    """
    
    def __init__(self, curve=None, generator=None):
        """
        Инициализация - выбор кривой, образующего элемента
        Возможно конструирование по готовой кривой и образующему
        """
        
        #
        # Для простоты буду использовать кривую из ГОСТ 34.10 
        #
        
        self._curve = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA'] if curve is None else curve
        self._generator = (self._curve.x, self._curve.y) if generator is None else generator
        self._modulus = self._curve.p
        
        trace('[EllipticCurveDiffieProcessor]', 
              f'Using group of order {self._modulus} over curve {self._curve.name}')
        
    
    def generate_private_key(self):
        """
        Генерирование закрытого ключа
        """
        
        return random.randint(int(2), int(self.order - 2))
    
    
    def get_public_key(self, private_key):
        """
        Получение открытого ключа по закрытому
        """
        
        return self.exponentiate(private_key, self._generator)
    
    
    def exponentiate(self, scalar, element):
        """
        Умножение точки на скаляр
        """
        
        return multiply_by_scalar(self._curve, scalar, element)
    
    
    def element_to_bytes(self, element):
        """
        Конвертация элемента группы в байтовое представление
        """
        
        return gost3410.pub_marshal(element)
    
    
    @property
    def group(self):
        """
        Получение группы
        """
        
        return self._curve
    
    
    @property
    def generator(self):
        """
        Получение образующего элемента
        """
        
        return self._generator
    
    
    @property
    def order(self):
        """
        Получение порядка группы
        """
        
        return self._modulus

----

In [12]:
cipher_suites = ['ECDHE_KUZNYECHIK_MGM_STREEBOG', 'DHE_AES_GCM_SHA256']

In [13]:
def resolve_cipher_suite(cipher_suite):
    """
    Получение примитивов по строковому идентификатору
    """
        
    #
    # Пока захардкодил это все
    #
        
    if cipher_suite == 'ECDHE_KUZNYECHIK_MGM_STREEBOG':
        return EllipticCurveDiffieProcessor, KuznyechikMGM, Streebog
        
    elif cipher_suite == 'DHE_AES_GCM_SHA256':
        return DiffieHellmanProcessor, AESGCM, SHA256
        
    raise ValueError(f'Unsupported cipher suite {cipher_suite}')

----

In [14]:
def hmac(key, *args, hasher=Streebog) -> bytes:
    """
    Вычисление HMAC (согласно Р 50.1.113 - 2016)
    Информационная технология
    КРИПТОГРАФИЧЕСКАЯ ЗАЩИТА ИНФОРМАЦИИ
    Криптографические алгоритмы, сопутствующие применению алгоритмов 
    электронной цифровой подписи и функции хэширования
    """
    
    def xor(lhs, rhs):
        return bytes([l ^^ r for l, r in zip(lhs, rhs)])
    
    IPAD = b'\x36' * 64
    OPAD = b'\x5c' * 64
    
    K_star = key + (b'\x00' * (512 - len(key)))
    
    inner_hash = hash_digest(hasher, xor(K_star, IPAD), *args)
    return hash_digest(hasher, xor(K_star, OPAD), inner_hash)

In [15]:
def test_hmac():
    """
    Р 50.1.113 - 2016
    Информационная технология
    КРИПТОГРАФИЧЕСКАЯ ЗАЩИТА ИНФОРМАЦИИ
    Криптографические алгоритмы, сопутствующие применению алгоритмов 
    электронной цифровой подписи и функции хэширования
    
    4.1.1 HMAC_GOSTR3411_2012_256
    
    Приложение А. Контрольные примеры
    """
    
    K = b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f' + \
        b'\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a\x1b\x1c\x1d\x1e\x1f'
    
    T = b'\x01\x26\xbd\xb8\x78\x00\xaf\x21\x43\x41\x45\x65\x63\x78\x01\x00'
    
    expected = b'\xa1\xaa\x5f\x7d\xe4\x02\xd7\xb3\xd3\x23\xf2\x99\x1c\x8d\x45\x34' + \
        b'\x01\x31\x37\x01\x0a\x83\x75\x4f\xd0\xaf\x6d\x7c\xd4\x92\x2e\xd9'
    
    assert hmac(K, T, hasher=Streebog) == expected

In [16]:
#
# Протестирую HMAC на контрольном примере из рекомендаций по стандартизации
#

test_hmac()

----

In [17]:
class GOST3410Signer(object):
    """
    Объект-создатель подписей по ГОСТ 34.10 - 2012
    """
    
    def __init__(self):
        """
        Инициализация кривой и ключевой пары
        """
        
        self._curve = gost3410.CURVES['id-tc26-gost-3410-2012-512-paramSetA']
        self._private_key = gost3410.prv_unmarshal(get_random_bytes(64))
        self._public_key = gost3410.public_key(self._curve, self._private_key)
        
        
    def sign(self, *args):
        """
        Подпись произвольного набора байтовых объектов
        """
        
        digest = hash_digest(Streebog, *args)
        return gost3410.sign(self._curve, self._private_key, digest)
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа
        """
        
        return self._public_key
    
    
    @property
    def private_key(self):
        """
        Получение закрытого ключа
        """
        
        return self._private_key

    
    @property
    def curve(self):
        """
        Получение используемой кривой
        """
        
        return self._curve
    
    
    @property
    def generator(self):
        """
        Получение образующего группы точек кривой
        """
        
        return self._curve.x, self._curve.y

In [18]:
def verify_signature(curve, public_key, signature, *args) -> bool:
    """
    Проверка подписи по ГОСТ 34.10 - 2012
    """
    
    digest = hash_digest(Streebog, *args)
    return gost3410.verify(curve, public_key, digest, signature)

----

## Реализация участников протокола

In [19]:
Offer = namedtuple('Offer', ['cipher_suite', 'group', 'generator'])

In [20]:
class GenericParticipant(object):
    """
    Класс-абстракция над участником протокола, который получает сертификат на свой открытый
    ключ и может осуществлять подпись сообщений
    """
    
    def __init__(self, ca, *, fake_data=None, demo_cert_error=False):
        """
        Инициализация - получение идентификатора, генерирование ключевой пары, 
        получение сертификата
        """
        
        self._id = uuid.uuid4()
        self._signer = GOST3410Signer()
        
        if fake_data is not None:
            #
            # Демонстрация неудачного выполнения протокола
            #
            
            if demo_cert_error:
                #
                # Тут демонстрация неудачи получения серификата
                #
                
                self._signer._public_key = fake_data
                self._certificate = ca.issue_certificate(self)
            else:
                #
                # Тут как бы сертификат и открытый ключ правильные, а закрытый ключ противник не угадал
                #
                
                self._certificate = ca.issue_certificate(self)
                self._signer._public_key = fake_data
        else:
            #
            # Стандартное выполнение
            #
            
            self._certificate = ca.issue_certificate(self)
        
        self._n = None
        
        trace('[GenericParticipant]', f'Certificate = {self._certificate}')
        
        
    def sign(self, *args):
        """
        Подписывание произвольных данных
        """
        
        return self._signer.sign(*args)
        
        
    @property
    def public_key(self) -> tuple[int, int]:
        """
        Получение открытого ключа подписи
        """
        
        return self._signer.public_key
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора
        """
        
        return self._id
    
    
    @property
    def certificate(self) -> Certificate:
        """
        Получение сертификата
        """
        
        return self._certificate
    
    
    @property
    def curve(self):
        """
        Получение кривой, используемой для подписи
        """
        
        return self._signer.curve
    
    
    def _get_point_for_schnorr_proof(self):
        #
        # Костыль, но все же работает
        #
        
        self._n = gost3410.prv_unmarshal(get_random_bytes(64))
        return multiply_by_scalar(self._signer.curve, 
                                  self._n, self._signer.generator)
    
    
    def _get_response(self, chellenge):
        return self._n + chellenge * self._signer.private_key

In [21]:
class TLS13Participant(GenericParticipant):
    """
    Обобщенный участник общения по TLS 1.3
    """
    
    def __init__(self, ca, *, fake_data=None, demo_cert_error=False):
        """
        Инициализация - по факту просто сохраняю ссылку на СА
        """
        
        super(TLS13Participant, self).__init__(ca, 
                                               fake_data=fake_data, 
                                               demo_cert_error=demo_cert_error)
        self._ca = ca
    
    
    def check_certificate(self, identifier: uuid.UUID, public_key, cert: Certificate) -> bool:
        """
        Проверка полученного сертификата
        """
        
        if self._ca.is_revoked(cert):
            trace('[TLS13Participant]', f'Revoked certificate')
            return False
            
        id_as_bytes = str(identifier).encode()
        pk_as_bytes = gost3410.pub_marshal(public_key)
        
        if not verify_signature(self._ca.curve, 
                                self._ca.public_key, 
                                cert.signature, 
                                id_as_bytes, 
                                pk_as_bytes):
            trace('[TLS13Participant]', f'Invalid certificate')
            return False
        
        return True
        
        
    @staticmethod
    def check_mac(key, mac, *args) -> bool:
        """
        Проверка имитовставки
        """
        
        return mac == hmac(key, *args)
    
    
    @staticmethod
    def check_signature(curve, public_key, signature, *args) -> bool:
        """
        Проверка подписи
        """
        
        return verify_signature(curve, public_key, signature, *args)

In [22]:
class TLS13Client(TLS13Participant):
    """
    Клиент для TLS 1.3
    """
        
    S2C = 0
    C2S = 1
    
    class AuthMode(Enum):
        """
        Режим аутентицикации
        """
        
        OWA = 1  # Односторонняя
        TWA = 2  # Двусторонняя
    
    
    def __init__(self, ca, cipher_suite, *, fake_data=None, demo_cert_error=False):
        """
        Инициализация - разбор набора примитивов
        """
        
        super(TLS13Client, self).__init__(ca, fake_data=fake_data, 
                                          demo_cert_error=demo_cert_error)
        
        self._key_exchange_type, self._cipher_type, self._hasher = resolve_cipher_suite(cipher_suite)
        self._cipher_suite = cipher_suite
        self._session_keys = {}
        
        trace('[TLS13Client]', f'Using cipher suite {cipher_suite}')
        
        
    def establish_connection(self, server, auth_mode: AuthMode) -> bool:
        """
        Установление соединения с сервером
        """
        
        key_exchange = self._key_exchange_type()
        
        ephemeral_dh_sk = key_exchange.generate_private_key()
        ephemeral_dh_pk = key_exchange.get_public_key(ephemeral_dh_sk)
        client_nonce    = get_random_bytes(64)
        
        trace('[TLS13Client]', f'Ephemeral DH public key: {ephemeral_dh_pk}')
        trace('[TLS13Client]', f'Nonce: {hexlify(client_nonce)}')
        
        offer = Offer(self._cipher_suite, 
                      key_exchange.group, 
                      key_exchange.generator)
        
        #
        # А теперь запрос на сервер
        #
        
        is_one_way = auth_mode == TLS13Client.AuthMode.OWA
        server_hello = server.get_hello(self.identifier, ephemeral_dh_pk, client_nonce, 
                                        offer, is_one_way)
        
        if server_hello is None:
            trace('[TLS13Client]', f'Server refused the connection')
            return False
        
        #
        # Распакую ответ
        #
        
        ephemeral_dh_server_pk, server_nonce, cipher_suite, c1, c2, c3, c4 = server_hello
        
        if cipher_suite != self._cipher_suite:
            trace('[TLS13Client]', f'Unsupported cipher suite {cipher_suite}')
            return False
        
        #
        # Общий секрет DH
        #
        
        ephemeral_dh_common = key_exchange.exponentiate(ephemeral_dh_sk, ephemeral_dh_server_pk)
        
        #
        # Вырабатываю ключи для симметричного шифрования и имитовставки
        #
        
        client_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_pk),
                                  client_nonce, str(offer).encode()])
        
        server_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_server_pk),
                                  server_nonce, cipher_suite.encode()])
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet])
        
        encryption_key, mac_key = HKDF(key_derivation_data, self._cipher_type.KEY_SIZE, 
                                       b'\x00' * 16, self._hasher, 2)
        
        #
        # Расшифровываю полученные пакеты
        #
        
        cipher = self._cipher_type(encryption_key)
        
        try:
            cert_requested     = cipher.decrypt(*c1)
            server_certificate = cipher.decrypt(*c2)
            server_signature   = cipher.decrypt(*c3)
            server_mac         = cipher.decrypt(*c4)
        except ValueError:
            trace('[TLS13Client]', f'Malformed server response')
            return False
        
        #
        # Проверяю полученнные данные
        #
        
        if is_one_way == cert_requested:
            #
            # Запросили сертификат в ходе односторонней аутентификации или наоборот
            #
            
            trace('[TLS13Client]', f'Invalid client certificate request')
            return False
        
        if not self.check_certificate(server.identifier, server.public_key, 
                                      pickle.loads(server_certificate)):
            trace('[TLS13Client]', f'Invalid server certificate')
            return False
        
        if not TLS13Participant.check_signature(server.curve, server.public_key, server_signature, 
                                                client_packet, server_packet, c1[1], c2[1]):
            trace('[TLS13Client]', f'Malformed server response')
            return False
        
        if not TLS13Participant.check_mac(mac_key, server_mac, 
                                          client_packet, server_packet, c1[1], c2[1], c3[1]):
            trace('[TLS13Client]', f'Malformed server response')
            return False
            
        #
        # Если требуется двусторонняя аутентификация, то зашлю на сервер сертификат
        #
        
        if not is_one_way:
            c5 = cipher.encrypt(pickle.dumps(self.certificate))
            
            signature = self.sign(client_packet, server_packet, c1[1], c2[1], c3[1], c4[1], c5[1])
            c6 = cipher.encrypt(signature)
            
            mac = hmac(mac_key, client_packet, server_packet, c1[1], c2[1], c3[1], c4[1], c5[1], c6[1])
            c7 = cipher.encrypt(mac)
            
            if not server.process_client_certificate(self, c5, c6, c7):
                trace('[TLS13Client]', f'Server refused the connection')
                return False
        
        #
        # Теперь я генерирую ключи шифрования
        #
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet, c1[1], c2[1], c3[1], c4[1]])
            
        self._session_keys[server.identifier] = HKDF(key_derivation_data, self._cipher_type.KEY_SIZE,
                                                     b'\x00' * 16, self._hasher, 2)
        
        trace('[TLS13Client]', f'Server accepted the connection')
        
        trace('[TLS13Client]', 
              f's -> c key: {hexlify(self._session_keys[server.identifier][TLS13Client.S2C])}')
        trace('[TLS13Client]', 
              f'c -> s key: {hexlify(self._session_keys[server.identifier][TLS13Client.C2S])}')
        
        return True
    
    
    def send_wait_receive_message(self, server, message: str) -> str:
        """
        Отправка сообщения на сервер и получение ответа
        """
        
        if server.identifier not in self._session_keys:
            trace('[TLS13Client]', f'Connection with server {server.identifier} is not established yet')
            return
        
        cipher = self._cipher_type(self._session_keys[server.identifier][TLS13Client.C2S])
        
        response = server._receive_message_and_respond(self, cipher.encrypt(message.encode()))
        if response is None:
            trace('[TLS13Client]', f'Malformed server response')
        
        try:
            cipher = self._cipher_type(self._session_keys[server.identifier][TLS13Client.S2C])
            return cipher.decrypt(*response).decode()
        except ValueError:
            trace('[TLS13Client]', f'Malformed server response')
            
            
    def change_keys(self, server):
        """
        Смена ключей по инициативе клиента
        """
        
        if server.identifier not in self._session_keys:
            trace('[TLS13Client]', f'Connection with server {server.identifier} is not established yet')
            return
        
        #
        # Меняю у себя
        #
        
        self._change_keys_internal(server.identifier)
        
        #
        # Меняю на сервере
        #
        
        server._change_keys_internal(self.identifier)
        
        
    def _change_keys_internal(self, server_id):
        s2c_key = HKDF(self._session_keys[server_id][TLS13Client.S2C], self._cipher_type.KEY_SIZE, 
                       b'\x00' * 16, self._hasher, 1)
        
        c2s_key = HKDF(self._session_keys[server_id][TLS13Client.C2S], self._cipher_type.KEY_SIZE, 
                       b'\x00' * 16, self._hasher, 1)
        
        trace('[TLS13Client]', f'New s -> c key: {hexlify(s2c_key)}')
        trace('[TLS13Client]', f'New c -> s key: {hexlify(c2s_key)}')
        
        self._session_keys[server_id] = (s2c_key, c2s_key)

In [23]:
class TLS13Server(TLS13Participant):
    """
    Сервер для TLS 1.3
    """
    
    CIPHER_TYPE            = 0
    HASHER                 = 1
    HANDSHAKE_ENC_KEY      = 2
    HANDSHAKE_MAC_KEY      = 3
    HANDSHAKE_VALUES       = 4
    HANDSHAKE_KEY_EXCHANGE = 5
    
    S2C = 0
    C2S = 1
    
    
    def __init__(self, ca, cipher_suites):
        """
        Инициализация - сохранение настроек
        """
        
        super(TLS13Server, self).__init__(ca)
        
        self._cipher_suites = cipher_suites
        self._session_keys  = {}
        self._mapped_states = {}
        
        
    def get_hello(self, client_id, ephemeral_dh_client_pk, client_nonce, offer, is_one_way):
        """
        Получение ServerHello
        """
        
        #
        # Провверяю, что поддерживается вообще набор примитивов
        #
        
        cipher_suite, group, generator = offer
        
        if cipher_suite not in self._cipher_suites:
            trace('[TLS13Server]', f'Unsupported cipher suite {cipher_suite}')
            return None
        
        trace('[TLS13Server]', f'Using cipher suite {cipher_suite} with client {client_id}')
        
        #
        # Получаю примитивы
        #
        
        key_exchange_type, cipher_type, hasher = resolve_cipher_suite(cipher_suite)
        key_exchange = key_exchange_type(group, generator)
        
        #
        # Вырабатываю ключи DH и nonce, после чего получаю общий секрет DH
        #
        
        ephemeral_dh_sk = key_exchange.generate_private_key()
        ephemeral_dh_pk = key_exchange.get_public_key(ephemeral_dh_sk)
        server_nonce    = get_random_bytes(64)
        
        trace('[TLS13Server]', f'Ephemeral DH public key: {ephemeral_dh_pk}')
        trace('[TLS13Server]', f'Nonce: {hexlify(client_nonce)}')
        
        ephemeral_dh_common = key_exchange.exponentiate(ephemeral_dh_sk, ephemeral_dh_client_pk)
        
        #
        # Вырабатываю ключи для симметричного шифрования и имитовставки
        #
        
        client_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_client_pk),
                                  client_nonce, str(offer).encode()])
        
        server_packet = b''.join([key_exchange.element_to_bytes(ephemeral_dh_pk),
                                  server_nonce, cipher_suite.encode()])
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet])
        
        encryption_key, mac_key = HKDF(key_derivation_data, cipher_type.KEY_SIZE, 
                                       b'\x00' * 16, hasher, 2)
        
        #
        # Рассчитываю нужные значения
        # 
        
        cipher = cipher_type(encryption_key)
        
        cert_request = bytes([not is_one_way])
        c1 = cipher.encrypt(cert_request)
        
        cert = pickle.dumps(self.certificate)
        c2 = cipher.encrypt(cert)
        
        signature = self.sign(client_packet, server_packet, c1[1], c2[1])
        c3 = cipher.encrypt(signature)
        
        mac = hmac(mac_key, client_packet, server_packet, c1[1], c2[1], c3[1])
        c4 = cipher.encrypt(mac)
        
        #
        # Тут будут сохранены нужные значения
        #
        
        self._mapped_states[client_id] = [None, None, None, None, None, None]
        
        #
        # Теперь выработаю ключи (если это односторонняя аутентификация)
        #
        
        if is_one_way:
            self._session_keys[client_id] = TLS13Server._generate_keys(key_exchange, cipher_type.KEY_SIZE, 
                                                                       hasher, ephemeral_dh_common, 
                                                                       client_packet, server_packet, 
                                                                       c1, c2, c3, c4)
        
        #
        # Сохраняю остальное и возвращаю клиенту
        #
        
        self._mapped_states[client_id][TLS13Server.CIPHER_TYPE]       = cipher_type
        self._mapped_states[client_id][TLS13Server.HASHER]            = hasher
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_ENC_KEY] = encryption_key
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_MAC_KEY] = mac_key
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_VALUES]  = (ephemeral_dh_common, client_packet, 
                                                                         server_packet, c1, c2, c3, c4)
        self._mapped_states[client_id][TLS13Server.HANDSHAKE_KEY_EXCHANGE] = key_exchange
        
        return ephemeral_dh_pk, server_nonce, cipher_suite, c1, c2, c3, c4
    
    
    def process_client_certificate(self, client, c5, c6, c7):
        """
        Проверка сертификата клиента
        """
        
        client_id = client.identifier
        
        #
        # Достану ранее сохраненные параметры
        #
        
        cipher_type    = self._mapped_states[client_id][TLS13Server.CIPHER_TYPE]
        hasher         = self._mapped_states[client_id][TLS13Server.HASHER]
        encryption_key = self._mapped_states[client_id][TLS13Server.HANDSHAKE_ENC_KEY]
        mac_key        = self._mapped_states[client_id][TLS13Server.HANDSHAKE_MAC_KEY]
        
        ephemeral_dh_common, client_packet, server_packet, c1, c2, c3, c4 = \
            self._mapped_states[client_id][TLS13Server.HANDSHAKE_VALUES]
        
        key_exchange   = self._mapped_states[client_id][TLS13Server.HANDSHAKE_KEY_EXCHANGE]
        
        #
        # Расшифрую полученный пакет
        #
        
        cipher = cipher_type(encryption_key)
        
        try:
            client_certificate = cipher.decrypt(*c5)
            client_signature   = cipher.decrypt(*c6)
            client_mac         = cipher.decrypt(*c7)
        except ValueError:
            trace('[TLS13Server]', f'Malformed client response')
            return False
        
        #
        # проверю все полученные данные
        #
        
        if not self.check_certificate(client.identifier, client.public_key, 
                                      pickle.loads(client_certificate)):
            trace('[TLS13Server]', f'Invalid client certificate')
            return False
        
        if not TLS13Participant.check_signature(client.curve, client.public_key, client_signature, 
                                                client_packet, server_packet, 
                                                c1[1], c2[1], c3[1], c4[1], c5[1]):
            trace('[TLS13Server]', f'Malformed client response')
            return False
        
        if not TLS13Participant.check_mac(mac_key, client_mac, client_packet, server_packet, 
                                          c1[1], c2[1], c3[1], c4[1], c5[1], c6[1]):
            trace('[TLS13Server]', f'Malformed client response')
            return False
        
        #
        # Все чудесно - генерирую ключи
        #
        
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet, c1[1], c2[1], c3[1], c4[1]])
            
        self._session_keys[client_id] = TLS13Server._generate_keys(key_exchange, cipher_type.KEY_SIZE, 
                                                                   hasher, ephemeral_dh_common, client_packet, 
                                                                   server_packet, c1, c2, c3, c4)
        
        return True
            
            
    def change_keys(self, client):
        """
        Смена ключей по инициативе сервера
        """
        
        if client.identifier not in self._session_keys:
            trace('[TLS13Client]', f'Connection with client {client.identifier} is not established yet')
            return
        
        #
        # Меняю у себя
        #
        
        self._change_keys_internal(client.identifier)
        
        #
        # Меняю на сервере
        #
        
        client._change_keys_internal(self.identifier)
        
    def _change_keys_internal(self, client_id):
        s2c_key = HKDF(self._session_keys[client_id][TLS13Client.S2C], 
                       self._mapped_states[client_id][TLS13Server.CIPHER_TYPE].KEY_SIZE, 
                       b'\x00' * 16, self._mapped_states[client_id][TLS13Server.HASHER], 1)
        
        c2s_key = HKDF(self._session_keys[client_id][TLS13Client.C2S], 
                       self._mapped_states[client_id][TLS13Server.CIPHER_TYPE].KEY_SIZE, 
                       b'\x00' * 16, self._mapped_states[client_id][TLS13Server.HASHER], 1)
        
        trace('[TLS13Client]', f'New s -> c key: {hexlify(s2c_key)}')
        trace('[TLS13Client]', f'New c -> s key: {hexlify(c2s_key)}')
        
        self._session_keys[client_id] = (s2c_key, c2s_key)
    
    
    def _receive_message_and_respond(self, client, encrypted_message):
        client_id = client.identifier
        
        try:
            cipher = self._mapped_states[client_id][TLS13Server.CIPHER_TYPE](
                self._session_keys[client_id][TLS13Server.C2S])
            
            message = cipher.decrypt(*encrypted_message).decode()
            print(f'Received message "{message}"')
            
            cipher = self._mapped_states[client_id][TLS13Server.CIPHER_TYPE](
                self._session_keys[client_id][TLS13Server.S2C])
            
            return cipher.encrypt(f'Hello, {message}!'.encode())
        except ValueError:
            trace('[TLS13Server]', f'Malformed client message')
    
    
    @staticmethod
    def _generate_keys(key_exchange, key_size, hasher, ephemeral_dh_common, 
                       client_packet, server_packet, c1, c2, c3, c4):
        key_derivation_data = b''.join([key_exchange.element_to_bytes(ephemeral_dh_common),
                                        client_packet, server_packet, c1[1], c2[1], c3[1], c4[1]])
            
        s2c_key, c2s_key = HKDF(key_derivation_data, key_size, 
                                b'\x00' * 16, hasher, 2)
        
        trace('[TLS13Server]', f's -> c key: {hexlify(s2c_key)}')
        trace('[TLS13Server]', f'c -> s key: {hexlify(c2s_key)}')
        
        return s2c_key, c2s_key

In [24]:
class TLS13CertificationAuthority(object):
    """
    Удостоверющий центр для TLS 1.3
    """
    
    def __init__(self):
        """
        Инициализация - генерирование ключевой пары
        """
        
        self._signer = GOST3410Signer()
        
        self._certificates = set()
        self._revoked_certificates = set()
        
        trace('[TLS13CertificationAuthority]', 
              f'Public key = {self._signer.public_key}')
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа
        """
        
        return self._signer.public_key
    
    
    @property
    def curve(self):
        """
        Получение кривой, используемой для подписи
        """
        
        return self._signer.curve
        
        
    def issue_certificate(self, participant: GenericParticipant) -> Certificate:
        """
        Выдача сертификата на открытый ключ
        """
        
        #
        # Проверю, что клиент действительно владеет закрытым ключом
        #
        
        if not self._check_participant(participant):
            trace('[TLS13CertificationAuthority]', 
                  f'Participant cannot prove a valid private key possession')
            
            return None
        
        #
        # Все чудесно, подписываю ключ и отдаю сертификат
        #
        
        id_as_bytes = str(participant.identifier).encode()
        pk_as_bytes = gost3410.pub_marshal(participant.public_key)
        
        cert = Certificate(participant.identifier, participant.public_key, 
                           self._signer.sign(id_as_bytes, pk_as_bytes))
        
        self._certificates.add(cert)
        
        trace('[TLS13CertificationAuthority]', 
              f'Issued certificate for {participant.identifier}. Total certificates: {len(self._certificates)}')
        
        return cert
    
    
    def revoke_certificate(self, participant: GenericParticipant):
        """
        Отзыв сертификата
        """
        
        #
        # А можно ли вообще отзывать сертификат?
        #
        
        if not self._check_participant(participant):
            trace('[TLS13CertificationAuthority]', 
                  f'Participant cannot prove a valid private key possession')
            
            return 
        
        if participant.certificate not in self._certificates:
            #
            # А отзывать то и нечего
            #
            
            return
            
        self._revoked_certificates.add(participant.certificate)
        self._certificates.discard(participant.certificate)
        
        trace('[TLS13CertificationAuthority]', 
              f'List of revoked certificates now has {len(self._revoked_certificates)} elements')
        
        
    def is_revoked(self, cert: Certificate):
        """
        Проверка на факт отзыва сертификата
        """
        
        return cert in self._revoked_certificates
    
    
    def _check_participant(self, participant: GenericParticipant) -> bool:
        Q = participant._get_point_for_schnorr_proof()
        c = gost3410.prv_unmarshal(get_random_bytes(64))
        t = participant._get_response(c)
        
        tP  = multiply_by_scalar(self._signer.curve, t, self._signer.generator)
        cPK = multiply_by_scalar(self._signer.curve, c, participant.public_key)
        
        return tP == add_points(self._signer.curve, Q, cPK)

----
## Демонстрация удачного выполнения протоколов

In [25]:
# Создание удостоверяющего центра
ca = TLS13CertificationAuthority()

[TRACE] [TLS13CertificationAuthority] Public key = (8981011505859747562952464463547903052499305639674098255156838657512754683992559421119253481649755530195345975115020433244438952047390758201187570841134091, 1209053376210063181051539730079355411100726623100157448834590546077337564352343447079429969234273224083846786752965397528062915265163429190848725154802544)


In [26]:
# Создание сервера
srv = TLS13Server(ca, cipher_suites)

[TRACE] [TLS13CertificationAuthority] Issued certificate for 7824c222-a414-4e79-bb16-fb8e9214c5a4. Total certificates: 1
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('7824c222-a414-4e79-bb16-fb8e9214c5a4'), public_key=(9980309512754189252713545941579480583146273843022512534344999392954678902910829444799316293411724699676145310082055542514259229434340854773665166544769851, 4643940927174956603998565051022568020560528070871700775199731979596592107101121055723984797381820306928314443899734177070049438036561787655235903160127790), signature=b'\x83\x921r\xff"\xf84lg\x86#\xe7n\x1cNH|\xdf\xc3\x80\xd0\x88\xf9\x1aF_\xab\x0b\x1f\xd1\xf0}\x951V\x07,\xb6\xa6\x86\xcev\x9e\xedj\xf19\xb6\x88\x84-\xde\xf4\x15\xb8C\xfb\x01\x1c\xffS{\xb0H\xfa4\x11\xda\xbcl\xc6\xd2\x8f\x81XK\xd5\x04\xf0\xac\xb8W9\xcd0\x8b\xe3~\xa7h!\xa6\xdaXY\xbf\xef\xc8\x1e\x1dE\x8d\xc1G\xd3ZU\x02Um\xf2\xfb\x04h\x94\x8f-qqd\xc3H\xdd\x97\xc2\xd6B')


In [27]:
# Первый валидный пользователь (использует ECDH, Кузнечик в режиме MGM и Стрибог)
alice = TLS13Client(ca, cipher_suite=cipher_suites[0])

[TRACE] [TLS13CertificationAuthority] Issued certificate for 8b96ae71-42c8-4e7d-b534-6a79ecafe265. Total certificates: 2
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('8b96ae71-42c8-4e7d-b534-6a79ecafe265'), public_key=(1212698964976100092184151934490439437745934281264327656406807261654011569005757968713579235716988197852036387255914149461591188148455629810626837882391975, 10239659052599157768710158126055471324757284785555476088551124645561246817250202413495088745323609061309654109209013688039318603473736442033031342821746063), signature=b"\xecCr\x8c4\xa3\rl\xa5\x90}\xb7R\xfb\xc4\xc0\x15\x19\xf9n\xff\x8e\x02\x832\xc3\xe9\x83\xe0\xd9\xd9_\xe68\xd4?\xbaJ\x92\x8eQ\x8fr\xb3\xd9q\x0cX\xdbe)=\xfa\xab\xb2\xfe\x99\xdd\x19w$0\xc4:\x91[\xa5:\x9fSK!A\xba1\x03\x87\xce\x0b'L\r\xdf\x0fkO\xcf\x80\x03\xea\xf0\xf2\xca1\xf2d?\xb6s\xde}-D+5__q\xf0\x96\xf2\x03\xef\x18\x8e\x1b\x9e\x1e\r\x00\xc9\xe8\xbeR\x03\xa1\x1f7")
[TRACE] [TLS13Client] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG


In [28]:
# Второй валидный пользователь (использует DH, AES в режиме GCM и SHA256)
bob = TLS13Client(ca, cipher_suite=cipher_suites[1])

[TRACE] [TLS13CertificationAuthority] Issued certificate for b32e690f-4b70-49b1-9500-670ef6541d48. Total certificates: 3
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('b32e690f-4b70-49b1-9500-670ef6541d48'), public_key=(4241720918938337264339764379478386118250464395517822091548680440278994418074891588088672007856375568914589337659503022153889926395346536330924957886492083, 12160634005822401347924784914798077394789258402373617326934630756254464543574597441473678801571843887639834974323319416874237360629371488516818810755011172), signature=b'\xfd\x12\xf6\xe0\x9cN\x00\xc9Q=?\xff\xf5\xb7\x84\x8d\\s\xeb\xaf8\x9dR/\xf8\x1dn\xe97\x13\xa0pC\xe6\xad\x90+\x18\xe3R\xd6\xde\xfa\xd5\xf5_\xcc\x8b\x1bB\x95:\x1a\n\xa9\xcd\x86\xdf\x13\xc6\x1d\xd5\x00\xec\xbdA\x908\xdc\x04\xaf\xf0H\x9c\x18/\xdd\xc5\xa7\\C\x82N\x07\x8bh\xe2L\xdf\x06\xce@\x86\x1f\x18+\xaf\xa4[\x9bc\xef>=\xcb\x88\xd8\xde0\xf1&\xb9\xa0/\x858\xe0mA_\xa56\x0b\x85\x92\x01\xfe\xa3')
[TRACE] [TLS13Client] Using cipher suite DHE_

In [29]:
# Устанавливаю соединение с односторонней аутентификацией
alice.establish_connection(srv, TLS13Client.AuthMode.OWA)

[TRACE] [EllipticCurveDiffieProcessor] Using group of order 13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006083527 over curve id-tc26-gost-3410-2012-512-paramSetA
[TRACE] [TLS13Client] Ephemeral DH public key: (4670640056286608044330424538534246253771554365876595822909219784701114622987149658485274154566553059562823436874207198786750869414102984940425131581258992, 1826728679804473613192004549377247688182403807082326190430262146986492332645228391536901488476038907321276739621251333401628058341256434307771223480631276)
[TRACE] [TLS13Client] Nonce: b'7b8b2804c7826fd2060240008e19ce9b8df6b9218b75d5ebe39bb2a69f9b2b1127a6ffc25608c4c457295cc1d65c753be08951ff67a86da49a1549625c0a28b5'
[TRACE] [TLS13Server] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG with client 8b96ae71-42c8-4e7d-b534-6a79ecafe265
[TRACE] [EllipticCurveDiffieProcessor] Using group of order 13407807929942597099574024998205846

True

In [30]:
# Устанавливаю соединение с двусторонней аутентификацией
bob.establish_connection(srv, TLS13Client.AuthMode.TWA)

[TRACE] [DiffieHellmanProcessor] Using group of order 13264407599174028581536607225753177421674144185395207597768831488216922524179885660636578020291880953982877241905840868828772039071236093056117443894995409
[TRACE] [TLS13Client] Ephemeral DH public key: 2686398506407487221671164288633769418688581515878647423468300152723697264975720831013961520873743155430991935909285786958508057518058762194899730759636879
[TRACE] [TLS13Client] Nonce: b'f503de6d9d20ad5095ab8c6dead1520b02ea3fb06e0f29977b75c6e0fea48e2a25a7147e86acf663657d54d3fe4f1fa6b4edbcf4ac69f35fd2d47c77cfdb5777'
[TRACE] [TLS13Server] Using cipher suite DHE_AES_GCM_SHA256 with client b32e690f-4b70-49b1-9500-670ef6541d48
[TRACE] [DiffieHellmanProcessor] Using group of order 13264407599174028581536607225753177421674144185395207597768831488216922524179885660636578020291880953982877241905840868828772039071236093056117443894995409
[TRACE] [TLS13Server] Ephemeral DH public key: 7651878075334616524040826679291974541566406646959809572670748

True

In [31]:
# Отправлю сообщение
alice.send_wait_receive_message(srv, 'world')

Received message "world"


'Hello, world!'

In [32]:
# Сменю ключи по инициативе клиента
alice.change_keys(srv)

[TRACE] [TLS13Client] New s -> c key: b'34eabc08421eca5af15b60aae893ee284ba743c48be990e3387fa556cbf7b285'
[TRACE] [TLS13Client] New c -> s key: b'b42a4e1bc2b9206f4635a432f6b20b49e39220ef379c1f546d99dd281e457d22'
[TRACE] [TLS13Client] New s -> c key: b'34eabc08421eca5af15b60aae893ee284ba743c48be990e3387fa556cbf7b285'
[TRACE] [TLS13Client] New c -> s key: b'b42a4e1bc2b9206f4635a432f6b20b49e39220ef379c1f546d99dd281e457d22'


In [33]:
# И еще раз отправлю сообщение, чтобы было видно, что ключи успешно поменялись
alice.send_wait_receive_message(srv, 'world')

Received message "world"


'Hello, world!'

In [34]:
# Отправлю сообщение другим клиентом
bob.send_wait_receive_message(srv, 'crypto')

Received message "crypto"


'Hello, crypto!'

In [35]:
# Смена ключей по инициативе сервера
srv.change_keys(bob)

[TRACE] [TLS13Client] New s -> c key: b'4940320336562b6f25e5fb1ec066cb88'
[TRACE] [TLS13Client] New c -> s key: b'1c6e17946140dc4ddc76f53870620255'
[TRACE] [TLS13Client] New s -> c key: b'4940320336562b6f25e5fb1ec066cb88'
[TRACE] [TLS13Client] New c -> s key: b'1c6e17946140dc4ddc76f53870620255'


In [36]:
# Все по-прежнему успешно отправляется
bob.send_wait_receive_message(srv, 'crypto')

Received message "crypto"


'Hello, crypto!'

----
## Отзыв сертификата

In [37]:
# Третий валидный пользователь
carol = TLS13Client(ca, cipher_suite=cipher_suites[0])

[TRACE] [TLS13CertificationAuthority] Issued certificate for f244875a-9261-4b24-a776-3ace4652b4b8. Total certificates: 4
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('f244875a-9261-4b24-a776-3ace4652b4b8'), public_key=(7332180649960455414152720595782932438128349311165073953063588281002794902423692678488713305493621244022439277910708576519467768857459572789540262860493027, 2013108013616755431015236494582883355149366081441143549121223086846598972859208041885299040497069021454704254965479961966019860616316195702548807340554526), signature=b'O\x15\x90\xd27\xf347\xa8\xe8!\x0e\xfd\x7f\x0c\xf6j\x1a\xe5Z\xde\x15\x9c&\xd4\x8a\xf1\xe7\x91\tU\xf07\x98M\xf8\xe6\x0f\xd6\x1b1\x98\xad<,\xf9\x12@d\xbb\xdb\x9f\xbf\x9f|\x07h\x05(\x9a\x02\x1a\xd7\x90d\xcc\xf3w\xb8\\\xc2T/\xad\xca\x8f\xb5;\xf5\xba\xa6\xcf4\x866\xe7\x02kmF\xdd\x13\x7f=\xd8\xc2Wy\x15\xa4L\x9c\x01\xd5\xd1D\xf8G vgt\x97\x1f4\x98\xb3\x8aL\xcb\x7f\xbdk\xe5h\xf7}\x19')
[TRACE] [TLS13Client] Using cipher suite ECDHE_KUZNYECHIK_M

In [38]:
# Отзову сертификат Кэрол
ca.revoke_certificate(carol)

[TRACE] [TLS13CertificationAuthority] List of revoked certificates now has 1 elements


In [39]:
# Попробую подключиться, используя двустороннюю аутентификацию
carol.establish_connection(srv, TLS13Client.AuthMode.TWA)

[TRACE] [EllipticCurveDiffieProcessor] Using group of order 13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006083527 over curve id-tc26-gost-3410-2012-512-paramSetA
[TRACE] [TLS13Client] Ephemeral DH public key: (9724536207862771395390889176279848125726787389834525728239632030977811965560425886943989746317701722789010867690445587576596392861467216895170715511157459, 9786219775277018391117402660968071153982186475361471231509250136668860506730826896223488369953802038837451339928966599959434223276082032415775192647509033)
[TRACE] [TLS13Client] Nonce: b'f7e89f7674368ea9fbb2a6d1646662bd382cf46be62c478a4177a5ca6f2d0fd02d0e32963ed4189f121fdc2f8b742f0740ada09235a11f7233598219110cdb33'
[TRACE] [TLS13Server] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG with client f244875a-9261-4b24-a776-3ace4652b4b8
[TRACE] [EllipticCurveDiffieProcessor] Using group of order 13407807929942597099574024998205846

False

----
## Демонстрация неудачного выполнения протоколов

In [40]:
# Тут Мэллори пытается установить подключение с двусторонней аутентификацией,
# используя сертификат не на свой ключ (я просто подменю тут открытый ключ для простоты)
mallory = TLS13Client(ca, cipher_suite=cipher_suites[0], 
                      fake_data=bob.public_key)

[TRACE] [TLS13CertificationAuthority] Issued certificate for 26bc2df0-edec-4893-8c6f-66c4cd34bcc7. Total certificates: 4
[TRACE] [GenericParticipant] Certificate = Certificate(id=UUID('26bc2df0-edec-4893-8c6f-66c4cd34bcc7'), public_key=(3158538158010658945645933767964375691319935291983347192725000225284542277150764880653515621045872602482631121107950502910295199136189098072624041162003107, 294512603360723271017034060001011741635191862038049684104958603075177838997648296241143906140957788300524551379372872229761966218997108658914036693189232), signature=b'y\x80\xb4I\x97/\xfd\x9e\x1d\xde\xb68j(p&J\x9eY2XXy\xc4\x16\x82\x00\x81\xfd7(\x13\xa2?\xafc\xb0\xe7\x98\xedsFbk\xadE$-\x8d\xad\xc2D\xba^\xf2\xde\x9d\xca\xa1\x0b\xbb\x99?\xa6:D)[F8\xa5ty\x1e\x92\xadC[.\xa5"\x9c\x7f\nc\xb5\xf8\xf0ki\x8aT\xe4\x97\x1a\x84\xf6\xad\x1e\x9a\xf3\x97\xf0\xa4r\xaf\xb3\xe8\xceP\xc0\xb4\xdc\xa5\xf5k/f\'1F\xda\x076/\x8e\x19`')
[TRACE] [TLS13Client] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG


In [41]:
mallory.establish_connection(srv, TLS13Client.AuthMode.TWA)

[TRACE] [EllipticCurveDiffieProcessor] Using group of order 13407807929942597099574024998205846127479365820592393377723561443721764030073546976801874298166903427690031858186486050853753882811946569946433649006083527 over curve id-tc26-gost-3410-2012-512-paramSetA
[TRACE] [TLS13Client] Ephemeral DH public key: (8806106685552990820221721265688773912442500873800940141198043219764062840354417005710533811795494833532118976430070178464031277581369239209618744783281633, 12653635929634351310197696946219175042726082773106086741711135411099063451488252337480425394861928227452603584868480706530278069449662856129868282082577626)
[TRACE] [TLS13Client] Nonce: b'648f583dd5e3ba485c13dfe306eb6625ac31ef83887032aeb962375816b47b660809c23d9268d5a3213c33e0bd1af5c20d910f58850625d80be5a39f41ec7adc'
[TRACE] [TLS13Server] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG with client 26bc2df0-edec-4893-8c6f-66c4cd34bcc7
[TRACE] [EllipticCurveDiffieProcessor] Using group of order 1340780792994259709957402499820584

False

In [42]:
# Мэллори теперь пытается получить сертификат на ключ Боба, не зная его закрытого ключа
mallory = TLS13Client(ca, cipher_suite=cipher_suites[0], 
                      fake_data=bob.public_key, 
                      demo_cert_error=True)

[TRACE] [TLS13CertificationAuthority] Participant cannot prove a valid private key possession
[TRACE] [GenericParticipant] Certificate = None
[TRACE] [TLS13Client] Using cipher suite ECDHE_KUZNYECHIK_MGM_STREEBOG
